# Merging Forex Data
This notebook takes all .csv files in the specified directory ('path' variable), keeps one column, names it accordingly, merges all date to one data frame. This is then saved as a pickle file for easy access.

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import pickle

In [2]:
path = '/home/robert/forex/data/currency'
extension = '.csv'
tomerge = []
import os
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(extension):
             tomerge.append(os.path.join(root, file))

In [3]:
print(tomerge)

['/home/robert/forex/data/currency/HISTDATA_COM_ASCII_USDJPY_M12009/DAT_ASCII_USDJPY_M1_2009.csv', '/home/robert/forex/data/currency/HISTDATA_COM_ASCII_USDCAD_M12010/DAT_ASCII_USDCAD_M1_2010.csv', '/home/robert/forex/data/currency/HISTDATA_COM_ASCII_EURUSD_M12010/DAT_ASCII_EURUSD_M1_2010.csv', '/home/robert/forex/data/currency/HISTDATA_COM_ASCII_USDCAD_M12007/DAT_ASCII_USDCAD_M1_2007.csv', '/home/robert/forex/data/currency/HISTDATA_COM_ASCII_USDCAD_M12008/DAT_ASCII_USDCAD_M1_2008.csv', '/home/robert/forex/data/currency/HISTDATA_COM_ASCII_USDJPY_M12010/DAT_ASCII_USDJPY_M1_2010.csv', '/home/robert/forex/data/currency/HISTDATA_COM_ASCII_GBPUSD_M12008/DAT_ASCII_GBPUSD_M1_2008.csv', '/home/robert/forex/data/currency/HISTDATA_COM_ASCII_EURUSD_M12009/DAT_ASCII_EURUSD_M1_2009.csv', '/home/robert/forex/data/currency/HISTDATA_COM_ASCII_GBPUSD_M12009/DAT_ASCII_GBPUSD_M1_2009.csv', '/home/robert/forex/data/currency/HISTDATA_COM_ASCII_USDJPY_M12008/DAT_ASCII_USDJPY_M1_2008.csv', '/home/robert/forex

In [4]:
datatypes = {'one': 'float32',
             'two': 'float32', 
             'three': 'float32',
             'four': 'float32',
             'five': 'int64'}

In [5]:
def parser(filename):
    folders = filename.split('/')
    info = folders[-1].split('.')
    mods = info[0].split('_')
    return '_'.join(mods[-3:])

In [6]:
def dayaverage(df):
    return df.groupby([df.index.month, df.index.day]).mean()

def houraverage(df):
    return df.groupby([df.index.month, df.index.day, df.index.hour]).mean()

In [11]:
def readforex(filename, datatypes, avg=None):
    colname = parser(filename)
    df = pd.read_csv(filename, header=None, names=[colname, 'two', 'three', 'four', 'five'], sep=',|;|\t\ ', engine='python', dtype=np.float32)
    df = df[[colname]]
    df.index = pd.to_datetime(df.index)
    if avg == 'day':
        return dayaverage(df)
    elif avg == 'hour':
        return houraverage(df)
    else:
        return df

In [12]:
dflist = []
for filename in tomerge:
    dflist.append(readforex(filename, datatypes))

In [13]:
data = pd.concat(dflist, axis=1)
outputfile = path + '/forex.pkl'
output = open(outputfile, 'wb')
pickle.dump(data, output)
output.close()